In [38]:
import requests
import pandas as pd 
import folium
from folium import FeatureGroup, features, CircleMarker, LayerControl
import numpy as np
from matplotlib import colors as mcolors
from haversine import haversine
from IPython.display import Markdown, display
from math import radians, sin, cos, asin, atan2, degrees
import json
import pickle

In [2]:
CP4 = 45.050940, 5.987655
finish =  48.395471, -4.434611
cp = folium.FeatureGroup(name='CP')
CircleMarker(CP4,color='purple').add_to(cp)
CircleMarker(finish,color='purple').add_to(cp)
bbox = cp.get_bounds()
bbox = [*bbox[0],*bbox[1]]
bbox

[45.05094, -4.434611, 48.395471, 5.987655]

In [11]:
query=\
"""
[out:json][timeout:900][maxsize:1073741824];
way[highway][ref~"^N"]({},{},{},{});
(._;>;);
out;"""

In [5]:
r = requests.get('http://overpass-api.de/api/interpreter?data='+query.format(*bbox))
print(r.status_code)
ores = r.json()

200


In [6]:
ores.get('remark')

'runtime error: Query ran out of memory in "query" at line 3. It would need at least 1036 MB of RAM to continue.'

In [7]:
def inBBOX(l,bbox):
    if bbox[0] < bbox[2] and bbox[1] < bbox[3]:
        return l[0] >= bbox[0] and l[0] <=  bbox[2] and l[1] >= bbox[1] and l[1] <= bbox[3]
    else:
        raise 'bbox not sorted'

In [8]:
racers = pd.read_pickle('racers.pkl')
mypath='./2019-08-12/Tracks/'
route={}
for racer in racers['Name']:
    try:
        ll =[l[0] for l in pd.read_pickle(mypath+racer+'.pkl').values.tolist()]
        route[racer] = ll
    except:
        print (racer)

In [10]:
mycolors = list(mcolors.cnames.keys())

In [9]:
fg = folium.FeatureGroup()
for i,name in enumerate(racers['Name']):
    if name not in ['Control_Car_1']:
        [CircleMarker(l,radius=2).add_to(fg) for l in route[name] if inBBOX(l,bbox)]
bbox = fg.get_bounds()
bbox = [*bbox[0],*bbox[1]]
bbox

[45.05096, -4.4346, 48.39545, 5.98761]

In [12]:
r = requests.get('http://overpass-api.de/api/interpreter?data='+query.format(*bbox))
print(r.status_code)
ores = r.json()

504


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
ores.get('remark')

Find the boundary for the lower part

In [13]:
bbox = fg.get_bounds()
bbox = [*bbox[0],*bbox[1]]
bbox1 = bbox
bbox1[2] = bbox1[0]+(bbox1[2]-bbox1[0])/2

In [14]:
fg1 = folium.FeatureGroup()
for name in racers['Name']:
    if name not in ['Control_Car_1']:
        [CircleMarker(l,radius=1).add_to(fg1) for l in route[name] if inBBOX(l,bbox1)]
bbox1a=fg1.get_bounds()

In [15]:
r = requests.get('http://overpass-api.de/api/interpreter?data='+query.format(*bbox1a[0],*bbox1a[1]))
print(r.status_code)
ores = r.json()

200


In [16]:
ores.get('remark')

In [17]:
ways =[e for e in ores['elements'] if e['type']=='way']
nodes = [e for e in ores['elements'] if e['type']=='node']
len(ways)

2834

In [35]:
nroutes = list(set([way['tags']['ref'] for way in ways]))
nroutes

['N 122',
 'N 2102',
 'N 47',
 'N 5',
 'N 88',
 'N 7',
 'N 147',
 'N 6',
 'N 2007',
 'N 82',
 'N 209',
 'N 481',
 'N 201',
 'N 102',
 'N 488',
 'N 346',
 'N 89',
 'N 145',
 'N 9',
 'N89',
 'N 510',
 'N 70',
 'N 2145',
 'N 520',
 'N 79',
 'N 141',
 'N 85',
 'N 78',
 'N 2043',
 'N 21',
 'N 87']

In [19]:
routenodes = []
for n in nroutes:
    nt = []
    for way in ways:
        if way['tags']['ref'] == n: 
            for nodeid in way['nodes']:
                for node in nodes:
                    if node['id'] == nodeid:
                        nt.append([node['lat'],node['lon']])
    routenodes.append(nt)

In [ ]:
bg =[]
for k, racer in racers.iterrows():
    try:
        if racer['Bib']=='CC1' or racer['Bib']=='CC2':
            continue
        for l in route[racer['Name']]:
            for j,routenode in enumerate(routenodes):
                found = False
                for r in routenode:  
                    d = haversine(l,r,'km')
                    if d < 2:
                        bg.append([j,nroutes[j],d,l,racer['Name'],racer['Bib']])
                        found = True
                        break
                if found: break
    except Exception as e:
        print(str(k)+' '+racer['Name']+' '+str(e),l,r)

Save list

In [39]:
with open('outfile.pkl', 'wb') as fp:
    pickle.dump(bg, fp)

In [46]:
bg

[]

In [ ]:
with open ('outfile.pkl', 'rb') as fp:
    bg = pickle.load(fp)

In [24]:
fgroute = [folium.FeatureGroup(name=name) for name in nroutes]

In [25]:
for i,fg in enumerate(fgroute):
    [CircleMarker(l,color='red',radius=1).add_to(fg) for l in routenodes[i]]

In [ ]:
m = folium.Map()
m.add_child(fg1)
fg1.layer_name = 'riders'
CircleMarker(CP4,color='purple').add_to(m)
[m.add_child(fg) for fg in fgroute]
folium.Polygon(bbox1)
m.fit_bounds(m.get_bounds())
LayerControl().add_to(m)
m

In [86]:
bbox1

[45.05096, 4.78272, 45.103198750000004, 5.98761]